In [1]:
#Model Weights
!cp "/content/drive/MyDrive/models/NER/NER.h5" "./"

In [2]:
#Tokenizer
!cp "/content/drive/MyDrive/models/NER/ner_tokenizer.json" "./"
!cp "/content/drive/MyDrive/models/NER/txt_tokenizer.json" "./"

In [3]:
import json
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, TimeDistributed, Dense
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences 
import numpy as np

In [4]:
def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size, classes):
    model = tf.keras.Sequential([
        Embedding(vocab_size, embedding_dim, mask_zero=True,
                  batch_input_shape=[batch_size,None]),
        Bidirectional(LSTM(units=rnn_units,
                           return_sequences=True)),
        TimeDistributed(Dense(rnn_units, activation='relu')),
        Dense(num_classes, activation="softmax")
    ])
    
    
    return model

In [5]:
with open('ner_tokenizer.json') as f:
  data = json.load(f)
  ner_tokenizer = tokenizer_from_json(data)

with open('txt_tokenizer.json') as f:
  data = json.load(f)
  txt_tokenizer = tokenizer_from_json(data)

In [6]:
ner_config = ner_tokenizer.get_config()
txt_config = txt_tokenizer.get_config()

In [7]:
ner_vocab = eval(ner_config["word_index"])
txt_vocab = eval(txt_config["word_index"])

In [8]:
ner_vocab

{'<OOV>': 1,
 'B-art': 12,
 'B-eve': 13,
 'B-geo': 3,
 'B-gpe': 9,
 'B-nat': 17,
 'B-org': 5,
 'B-per': 7,
 'B-tim': 4,
 'I-art': 14,
 'I-eve': 15,
 'I-geo': 10,
 'I-gpe': 16,
 'I-nat': 18,
 'I-org': 8,
 'I-per': 6,
 'I-tim': 11,
 'O': 2}

In [9]:
vocab_size = len(txt_vocab) + 1
embedding_dim = 64
rnn_units = 100
BATCH_SIZE=1
num_classes = len(ner_vocab) + 1
model = build_model_bilstm(
    vocab_size = vocab_size,
    embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE,
    classes=num_classes)

In [10]:
model.load_weights("NER.h5")

In [11]:
ner_keys = list(ner_vocab.keys())
ner_values = list(ner_vocab.values())

In [12]:
def get_result(result):
  result_index = []
  actual_result = []
  for i in range(result.shape[1]):
    ner_pred = np.argmax(result[0][i])
    result_index.append(ner_pred)
  for r in result_index:
    pos = ner_values.index(r)
    actual_result.append(ner_keys[pos])
  return actual_result

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (1, None, 64)             2523072   
                                                                 
 bidirectional (Bidirectiona  (1, None, 200)           132000    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (1, None, 100)           20100     
 ibuted)                                                         
                                                                 
 dense_1 (Dense)             (1, None, 19)             1919      
                                                                 
Total params: 2,677,091
Trainable params: 2,677,091
Non-trainable params: 0
_________________________________________________________________


In [14]:
def preprocess_text(sent):
  sent = sent.split(" ")
  tokenized= txt_tokenizer.texts_to_sequences(sent)
  tokenized_arr = []
  for i in tokenized:
    tokenized_arr.append(i[0])
  return np.array(tokenized_arr).reshape(1, len(tokenized_arr))

In [15]:
def make_prediction(text, model= model):
  text= text
  tokenized_text = preprocess_text(text)
  result= model.predict(tokenized_text)
  pred = get_result(result)
  return pred

In [16]:
text = "I want to visit New York"
make_prediction(text)

['O', 'O', 'O', 'O', 'B-geo', 'I-geo']

The model was able to detect that New York as a "Geographical" Location

In [17]:
text2 = "My name is Mahesh"
make_prediction(text2)

['O', 'O', 'O', 'B-per']

The model was able to detect Mahesh as a "Person".

In [18]:
text3 = "Apple is a big company."
make_prediction(text3)

['B-org', 'O', 'O', 'O', 'B-org']

The model was able to detect "Apple" as an "Organization". Also, it classified  "Company" as an "Organization".


In [19]:
text4 = "I ordered lights from Amazon"
make_prediction(text4)

['O', 'O', 'O', 'O', 'B-org']

The model was able to detect "Amazon" as an "Organization".

In [20]:
text5 = "Meet me in the morning"
make_prediction(text5)

['O', 'O', 'O', 'O', 'B-tim']

The model was able to detect "morning" as an "Time".

In [21]:
text6 = "Jeff would you like to have some coffee?"
make_prediction(text6)

['B-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

The model was able to detect "Jeff" as an "Person"